In [1]:
import os
import sys
sys.path.append("..")
import phage_init

In [2]:
#load the saved matrices
import pickle
train_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"train_x.p"), "rb" ) )
test_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"test_x.p"), "rb" ) )
train_Y=pickle.load(open( os.path.join(phage_init.data_dir,"train_y.p"), "rb" ) )
test_Y=pickle.load(open( os.path.join(phage_init.data_dir,"test_y.p"), "rb" ) )
mean_total=pickle.load(open( os.path.join(phage_init.data_dir,"mean.p"), "rb" ) )
std_total=pickle.load(open( os.path.join(phage_init.data_dir,"std.p"), "rb" ) )

In [3]:
di_train     = train_X_total[:,0:400]
tri_train    = train_X_total[:,400:8400]
di_sc_train  = train_X_total[:,8400:8449] 
tri_sc_train = train_X_total[:,8449:8792]
tt_train     = train_X_total[:,8792:]

In [4]:
di_test     = test_X_total[:,0:400]
tri_test    = test_X_total[:,400:8400]
di_sc_test  = test_X_total[:,8400:8449] 
tri_sc_test = test_X_total[:,8449:8792]
tt_test     = test_X_total[:,8792:]

In [5]:
mean_di     = mean_total[0:400]
mean_tri    = mean_total[400:8400]
mean_di_sc  = mean_total[8400:8449] 
mean_tri_sc = mean_total[8449:8792]
mean_tt     = mean_total[8792:]

In [ ]:
std_di     = std_total[0:400]
std_tri    = std_total[400:8400]
std_di_sc  = std_total[8400:8449] 
std_tri_sc = std_total[8449:8792]
std_tt     = std_total[8792:]

In [ ]:
#import keras and numpy
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:

train_X =numpy.concatenate((tri_sc_train,tt_train),axis=1)
test_X  =numpy.concatenate((tri_sc_test,tt_test),axis=1)
#train_X=di_sc_train
#test_X=di_sc_test
model_name='tri_sc_p.h5'
print(train_X.shape)
print(test_X.shape)

(25000, 351)
(675, 351)


In [ ]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16391753317458432191
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15540864114754508418
physical_device_desc: "device: XLA_CPU device"
]


In [ ]:
print(train_X.shape[1])
f_num=train_X.shape[1]


351


In [ ]:
#model with 2 leyers of 100 LSTM neurons
model = Sequential()
opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.add(Dense(f_num, input_dim=f_num, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 351)               123552    
_________________________________________________________________
dropout_1 (Dropout)          (None, 351)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               70400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total para

In [ ]:
model.fit(train_X, train_Y, epochs=200,verbose=1, batch_size=3000)

Epoch 1/200
25000/25000 [==============================] - 1s 59us/step - loss: 2.0977 - acc: 0.2611
Epoch 2/200
25000/25000 [==============================] - 0s 19us/step - loss: 1.5671 - acc: 0.4965
Epoch 3/200
25000/25000 [==============================] - 0s 19us/step - loss: 1.2088 - acc: 0.6056
Epoch 4/200
25000/25000 [==============================] - 0s 19us/step - loss: 1.0203 - acc: 0.6676
Epoch 5/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.8908 - acc: 0.7040
Epoch 6/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.7965 - acc: 0.7378
Epoch 7/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.7358 - acc: 0.7546
Epoch 8/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.6787 - acc: 0.7704
Epoch 9/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.6256 - acc: 0.7888
Epoch 10/200
25000/25000 [==============================] - 0s 18us/step - loss: 0.5883 - a

25000/25000 [==============================] - 0s 19us/step - loss: 0.1846 - acc: 0.9132
Epoch 82/200
25000/25000 [==============================] - 0s 20us/step - loss: 0.1806 - acc: 0.9126
Epoch 83/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.1804 - acc: 0.9116
Epoch 84/200
25000/25000 [==============================] - 1s 21us/step - loss: 0.1803 - acc: 0.9136
Epoch 85/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.1827 - acc: 0.9108
Epoch 86/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.1809 - acc: 0.9124
Epoch 87/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.1792 - acc: 0.9132
Epoch 88/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.1775 - acc: 0.9124
Epoch 89/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.1799 - acc: 0.9128
Epoch 90/200
25000/25000 [==============================] - 0s 19us/step - loss: 0.1777 - acc: 

In [ ]:
scores = model.evaluate(test_X, test_Y, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
#you can save the as a file so you dont have to train it every time
model.save( os.path.join(phage_init.model_dir,model_name))

In [ ]:
from sklearn.metrics import classification_report


test_Y_index = test_Y.argmax(axis=1) # Convert one-hot to index
test_Y_predicted = model.predict_classes(test_X)
labels_names=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
             "Tail shaft","Collar","Head-Tail joining"]
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
#print (test_Y_index)

In [ ]:
from collections import Counter
zz=Counter(test_Y_index)
sample_w=[zz[i] for i in range(0,10,1)]
print(zz)
print(sample_w)
print()

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
plt.show()
CM=confusion_matrix(test_Y_index, test_Y_predicted)
CM_n=CM/numpy.array(sample_w)[:,None]
scale_up=1.3
plt.figure(figsize=[6.4*scale_up, 4.8*scale_up])
plt.imshow(CM_n, interpolation='nearest')
plt.title('CM')
plt.colorbar()
tick_marks = numpy.arange(len(labels_names))
plt.xticks(tick_marks, labels_names, rotation=90)
plt.yticks(tick_marks, labels_names)
fmt = '.2f'
for i, j in itertools.product(range(CM_n.shape[0]), range(CM_n.shape[1])):
        plt.text(j, i, format(CM_n[i, j], fmt),horizontalalignment="center",verticalalignment='center',
                color="white" if CM_n[i, j] < 0.25 else "black")
plt.show()

In [ ]:
print(numpy.where(~train_X.any(axis=0))[0])

In [ ]:
numpy.set_printoptions(threshold=numpy.nan)
print(std_total)